# __Telco Customer Churn Evaluation and Analysis__

> ## Question

>> To what extent can telco customer churn rate be reduced, using the IBM telco customer churn dataset?

> ## Introduction

>> Customer churn rate refers to the proportion of customers who cancel their subscription to a company provided service. Reducing customer churn rate is essential, as often times it costs more money to attract new customers compared to maintaining the ones currently subscribed. If one is able to figure out a way to reduce customer churn, there would most likely be an increase in the profits for the company. The goal of this note book will go through analyzing and evaluating what features from the data are important and providing suggestions as to how the telco company may reduce its customer churn rate. 


> ## Evaluation

>> __Note:__ Loading the libraries and loading the data

In [1]:
import pandas as pd
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.preprocessing import StandardScaler
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.metrics import classification_report, accuracy_score, roc_auc_score, confusion_matrix, ConfusionMatrixDisplay, precision_recall_curve, precision_score, recall_score, f1_score
from sklearn.neighbors import KNeighborsClassifier
from matplotlib.pyplot import subplots
import seaborn as sns
import matplotlib.pyplot as plt
from imblearn.over_sampling import SMOTE

In [2]:
data = pd.read_csv('customer_data.csv')

>> According to the provided dataset, the company currently has a churn rate of 26.54%. This means that almost 27% of customers left the company at the end of the quarter. As stated above, our goal is to figure out what features are currently contributing the most to the churn rate, and provide potential solutions as to what could be done.

In [3]:
total_customers = data.shape[0]
churned_customers = data[data['Churn'] == 1].shape[0]
churn_rate = churned_customers / total_customers

>> Our first step is to get a better understanding of the data that we are working with. The proceeding code shows that the dataset consists of 21 parameters with 7043 observations. A good number of the columns have 'object' as their data type which will need to be dealt with before we are able to move onto any modelling. 

In [4]:
print(f"Dataset Shape: {data.shape}")
print(f"Columns: {data.columns.tolist()}")
print(data.dtypes)



Dataset Shape: (7043, 21)
Columns: ['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents', 'tenure', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaperlessBilling', 'PaymentMethod', 'MonthlyCharges', 'TotalCharges', 'Churn']
customerID           object
gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
StreamingTV          object
StreamingMovies      object
Contract             object
PaperlessBilling     object
PaymentMethod        object
MonthlyCharges      float64
TotalCharges         object
Churn                object
dtype: object


>> A quick check as to whether any of the columns contain null values show that none of the columns contain any. However, this does not mean that all the entries are useful. For example, even if a column has no null values it could still contain entries such as " ", which will not be classified as a null value but will cause problems down the road when we move onto setting up the data for modelling. 

In [5]:
print(data.isnull().sum())

customerID          0
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
dtype: int64


>> I first wanted to see how many individual services each of the observations has, as the number of services is related to the overall total charge, which in turn is one of the parameters, that I believe, will be most impactful to determining the churn rate. I did this before dealing with categorical columns as the dataset will be easier to manipulate right now.

In [6]:
paying_cols = ['MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

for c in paying_cols:
    print(data[c].unique())

data['TotalSubscriptions'] = 0

for idx, row in data.iterrows():
    total_services = 0 
    for col in paying_cols:
        if row[col] == 'Yes' or row[col] == 'DSL' or row[col] == 'Fiber optic':
            total_services += 1

    data.at[idx, 'TotalSubscriptions'] = total_services


['No phone service' 'No' 'Yes']
['DSL' 'Fiber optic' 'No']
['No' 'Yes' 'No internet service']
['Yes' 'No' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']
['No' 'Yes' 'No internet service']


>> We then move onto changing columns that contain categorical values, to contain numeric values. I first mapped out the unique responses from each categorical column before using the ``` pd.get_dummies()``` function. The reason I chose to use one-hot encoding is because it is nominal, where as ranking categories is ordinal, which could mask true interactions between variables. 

In [7]:
data_mapping = {
    'Yes': 1,
    'No': 0,
    'No phone service': 0,
    'Female': 1,
    'Male': 0
}

internet_dummies = pd.get_dummies(data['InternetService'], prefix='InternetService', drop_first=True)
security_dummies = pd.get_dummies(data['OnlineSecurity'], prefix='OnlineSecurity', drop_first=True)
backup_dummies = pd.get_dummies(data['OnlineBackup'], prefix='OnlineBackup', drop_first=True)
protection_dummies = pd.get_dummies(data['DeviceProtection'], prefix='DeviceProtection', drop_first=True)
support_dummies = pd.get_dummies(data['TechSupport'], prefix='TechSupport', drop_first=True)
tv_dummies = pd.get_dummies(data['StreamingTV'], prefix='StreamingTV', drop_first=True)
movie_dummies = pd.get_dummies(data['StreamingMovies'], prefix='StreamingMovies', drop_first=True)
contract_dummies = pd.get_dummies(data['Contract'], prefix='Contract', drop_first=True)
payment_dummies = pd.get_dummies(data['PaymentMethod'], prefix='PaymentMethod', drop_first=True)

og_col_name = ['InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies', 'Contract', 'PaymentMethod']

data['Partner'] = data['Partner'].map(data_mapping)
data['Dependents'] = data['Dependents'].map(data_mapping)
data['PhoneService'] = data['PhoneService'].map(data_mapping)
data['MultipleLines'] = data['MultipleLines'].map(data_mapping)
data['gender'] = data['gender'].map(data_mapping)
data['PaperlessBilling'] = data['PaperlessBilling'].map(data_mapping)
data["Churn"] = data['Churn'].map(data_mapping)

data = data.drop(og_col_name, axis=1)

data = pd.concat([data, internet_dummies, security_dummies, backup_dummies, protection_dummies, support_dummies, tv_dummies, movie_dummies, contract_dummies, payment_dummies], axis=1)



In [8]:
data.head()

,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,PaperlessBilling,MonthlyCharges,...,TechSupport_Yes,StreamingTV_No internet service,StreamingTV_Yes,StreamingMovies_No internet service,StreamingMovies_Yes,Contract_One year,Contract_Two year,PaymentMethod_Credit card (automatic),PaymentMethod_Electronic check,PaymentMethod_Mailed check
0,7590-VHVEG,1,0,1,0,1,0,0,1,29.85,...,False,False,False,False,False,False,False,False,True,False
1,5575-GNVDE,0,0,0,0,34,1,0,0,56.95,...,False,False,False,False,False,True,False,False,False,True
2,3668-QPYBK,0,0,0,0,2,1,0,1,53.85,...,False,False,False,False,False,False,False,False,False,True
3,7795-CFOCW,0,0,0,0,45,0,0,0,42.30,...,True,False,False,False,False,True,False,False,False,False
4,9237-HQITU,1,0,0,0,2,1,0,1,70.70,...,False,False,False,False,False,False,False,False,True,False


>> As mentioned previously, just because a column does not have any null values, does not mean that every entry in that column contains useful information. This is seen most obviously from the 'TotalCharges' column. Some of the values said column were just empty quotation marks, ' ', which caused problems down the line when trying to scale the data before performing further tests. To combat this, I replaced all entries that were empty quotations with null values, converted the entry values type to floats, calculated the mean, and then replaced all the null values with the calculated mean, before finally permanently converting the 'TotalCharges' column to type float.

In [9]:
data = data.replace(r'^\s*$', np.nan, regex=True)

total_charges_mean = data['TotalCharges'].astype(float).mean()
data['TotalCharges'].fillna(total_charges_mean, inplace=True)
data['TotalCharges'] = data['TotalCharges'].astype(float)

/var/folders/bz/m9n4tlrx3jd3br1gzvg51q_00000gn/T/ipykernel_32822/4280112463.py:4: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  data['TotalCharges'].fillna(total_charges_mean, inplace=True)


>> The tenure column was also one that required further modification. Instead of just letting each observation lie separately, I decided to group them up by the amount of years there had been subscribed to the comapny. I did this by creating bins, assigning a value to each bin - such as '0-1 year' - and then using ```pd.cut()``` to divide the tenure column into the created bins, assigning each row to the proper bin based on its tenure value. This creates a new column in the dataset where each customer falls into a different bin category based on how the value they entered under the 'tenure' column. 

In [10]:
t_range = [0, 12, 36, 60, float('inf')]
years = ['0-1 year', '1-3 years', '3-5 years', '5+ years']

data['tenure_group'] = pd.cut(data['tenure'], bins=t_range, labels=years, right=False)


>> After this, I took a slight detour, before moving on to creating actual mdoels, to create some graphs in order to visualize the distribution churn rate and specific columns. 

__Note:__ I created the visualizations in R as I personally like the aesthetic more than those created in Python.

In [11]:
data.to_csv('cleaned_data.csv', index=False)

In [12]:
print(data['Churn'].value_counts())

Churn
0    5174
1    1869
Name: count, dtype: int64


<div style="display: flex; justify-content: space-around;">
    <img src="Graphs/churnxgender.jpeg" alt="Image 1" width="450">
    <img src="Graphs/Tenuregroupxchurn.jpeg" alt="Image 2" width="450">
</div>

<div style="display: flex; justify-content: space-around;">
    <img src="Graphs/churncount.jpeg" alt="Image 1" width="450">
</div>

__Figure I.__ Bar chart showing the relationship between gender and churn status

__Figure II.__ Bar chart showing the relationship between the four different bins (0-1 year, 1-3 years, 3-5 years, 5+ years) and churn status

__Figure III.__ Bar chart showing the count of customers who stayed with the company and customers who cancelled the subscription service

>> Figure I shows that there is little difference between the churn rate of male and female customers, meaning that when it comes to modelling, gender most likely will not be a feature of high importance. Figure II, on the other hand, shows a clear relationship between the tenure group the churn status. The graph clearly shows that the highest amount of churn comes from subscribers who have been with the one year or less. As the tenure window gets longer and longer, the churn amount also drops. From this, we can predict that total charges will be a feature of high importance when it comes to modelling and may be something that we want to deal with moving forward. One interesting thing to note is that, as tenure is just the amount of time that a customer has been subscribed to the company, there will be less that we can do to directly impact the amount of time someone has been subscribed. What I mean by this is that, unlike other columns such as "Total Charges" -which we can directly impact by deciding whether we should increase or decrease the charges-, there is no way for us to directly extend a customers tenure to 5+ years. 

>## Modelling

>> From here on out, I will be discussing the models and the methods that I used to figure out which features are important and what things to change in order to improve model accuracy.

>> First, I created dataset X and y. X is derived from the original dataset that we have been working on thoughout the entirety of this notebook. However, X is not a complete copy. I removed certain columns that I deemed to have absolutely no importance in modelling, such as "ID", or redundant columns that were created in order to facilitate the generation of the visualizations and of course the response variable 'Churn'. Dataset y only contains the 'Churn' column from the original dataset. 

In [13]:
data.columns

Index(['customerID', 'gender', 'SeniorCitizen', 'Partner', 'Dependents',
       'tenure', 'PhoneService', 'MultipleLines', 'PaperlessBilling',
       'MonthlyCharges', 'TotalCharges', 'Churn', 'TotalSubscriptions',
       'InternetService_Fiber optic', 'InternetService_No',
       'OnlineSecurity_No internet service', 'OnlineSecurity_Yes',
       'OnlineBackup_No internet service', 'OnlineBackup_Yes',
       'DeviceProtection_No internet service', 'DeviceProtection_Yes',
       'TechSupport_No internet service', 'TechSupport_Yes',
       'StreamingTV_No internet service', 'StreamingTV_Yes',
       'StreamingMovies_No internet service', 'StreamingMovies_Yes',
       'Contract_One year', 'Contract_Two year',
       'PaymentMethod_Credit card (automatic)',
       'PaymentMethod_Electronic check', 'PaymentMethod_Mailed check',
       'tenure_group'],
      dtype='object')

In [14]:
X = data.drop(["Churn", 'customerID', 'tenure_group'], axis=1)
y = data['Churn']

>> Now that we have our designated datasets, I start off by first running a variance inflation factor test to see which, if any of the features are displaying linearity or multicollinearity. Normally, features with VIF values greater than 5 need to be considered to be linear to atleast one other feature, however, we need to first scale the values of X. 

In [15]:
scaler = StandardScaler()

X_scaled = scaler.fit_transform(X)

In [16]:
print(X_scaled)

[[ 1.00955867 -0.43991649  1.03453023 ... -0.52504733  1.40641839
  -0.54480692]
 [-0.99053183 -0.43991649 -0.96662231 ... -0.52504733 -0.71102597
   1.83551265]
 [-0.99053183 -0.43991649 -0.96662231 ... -0.52504733 -0.71102597
   1.83551265]
 ...
 [ 1.00955867 -0.43991649  1.03453023 ... -0.52504733  1.40641839
  -0.54480692]
 [-0.99053183  2.27315869  1.03453023 ... -0.52504733 -0.71102597
   1.83551265]
 [-0.99053183 -0.43991649 -0.96662231 ... -0.52504733 -0.71102597
  -0.54480692]]


>> From the table of VIF values, it can be seen that even though I used one-hot encoding where we drop one of the categories, there is still collineaity amongst the features. Almost all of the features that have a VIF value greater than 5 come from a one-hot encoded feature. For now, I will leave the dataset as is and contineu onto actually creating a model. The main model that I have decided to use for this project is Random Forest. One of the reasons I decided to use this machine learning model is because it handles multicollinearity which appears to be an issue with this currnet dataset as can be seen from the VIF table.

In [17]:
vif_data = pd.DataFrame()
vif_data['Feature'] = X.columns
vif_data['VIF'] = [variance_inflation_factor(X_scaled, i) for i in range(X_scaled.shape[1])]
print(vif_data)

/Users/bengrigsby/.pyenv/versions/3.12.2/lib/python3.12/site-packages/statsmodels/stats/outliers_influence.py:197: RuntimeWarning: divide by zero encountered in scalar divide
  vif = 1. / (1. - r_squared_i)


                                  Feature         VIF
0                                  gender    1.002151
1                           SeniorCitizen    1.153352
2                                 Partner    1.462800
3                              Dependents    1.384160
4                                  tenure    7.304521
5                            PhoneService   34.862059
6                           MultipleLines         inf
7                        PaperlessBilling    1.209019
8                          MonthlyCharges  865.053343
9                            TotalCharges   10.541130
10                     TotalSubscriptions         inf
11            InternetService_Fiber optic  148.263480
12                     InternetService_No         inf
13     OnlineSecurity_No internet service         inf
14                     OnlineSecurity_Yes         inf
15       OnlineBackup_No internet service         inf
16                       OnlineBackup_Yes         inf
17   DeviceProtection_No int

>> I first split the data into the training set and the test set. The training set contains 80% of the data while the test set contains the remaining 20%.

In [18]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.2, random_state=30)

>> I create the random forest model, setting the number of trees in the forest to 100, the maximum depth of each decision tree to 10, the minimum number of samples required per leaf to be 4, and the minimum number of samples required to split an internal node to be 10. I have yet to hypertune the parameters, and I just first wanted to create a model of the tree with slightly parameter values just to get a sense of the model. I then fit the random forest to the training dataset.

In [19]:
og_rf_model = RandomForestClassifier(
    n_estimators= 100,
    max_depth=10,
    min_samples_leaf=4,
    min_samples_split=10,
    random_state=30,
)

og_rf_model.fit(X_train, y_train)

RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                       random_state=30)

>> Now onto the fun stuff. Putting the random forest model into action and seeing how well it works using the test set. 

In [20]:
og_y_pred = og_rf_model.predict(X_test)
og_y_pred_proba = og_rf_model.predict_proba(X_test)[:,1]

>> The accuracy of the model is calculated, along side a classification report -which include important indicators such as the f1-score-, and the ROC-AUC curve. All these different metrics tell us a slightly different story about the effectiveness of our random forest model. The __accuracy of our model__ is almost and 80%. The __precision level__, which measures how accurately the model's positive predictions where, lies around 81% for predicting which people are not going to churn, and a 71% for people who are going to churn. The model had a __recall score__ of around 92% for non-churners. This means that out of all of the actual people who did not churn, the model correctly predicted 92% of them. When it came to the predicting the actual churners, the model only correctly predicted 48% of them. Ideally, we would want this model to be higher as this is the most important aspect that the model should aim to get right. The model has an __f1-score__ of 86% for class 0 meaning that there is a reasonable balance between precision and recall when the model is predicting for non-churners. However, there is definitely room for improvement for the model, as the f1-score it has when predicting for actual churners is only a 57%. This means that there is an imbalance between the precision and recall of the model, which is also shown by the corresponding values. The ROC-AUC score, which tells us how well a model distinguishes between classes, lies around an 84%. Though this number may seem relatively high and indicate an accurate model, when we consider the other values given by the classification report, there is much left to be desired from our random forest model. This is because even though the model did well on predicting non-churners, the performace really dropped when it came to predicting churners. As churners are the more important group, despite the model having an AUC score of ~84%, we should not look too closely at that singular value, and instead aim to improve the model's precision and recall score when it comes to predicting people who churn.

In [21]:
accuracy = accuracy_score(y_test, og_y_pred)
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n {classification_report(y_test, og_y_pred)}")
roc_auc = roc_auc_score(y_test, og_y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

Accuracy: 0.7906316536550745
Classification Report:
               precision    recall  f1-score   support

           0       0.81      0.92      0.86      1001
           1       0.71      0.48      0.57       408

    accuracy                           0.79      1409
   macro avg       0.76      0.70      0.71      1409
weighted avg       0.78      0.79      0.78      1409

ROC-AUC Score: 0.8386943938414527


In [22]:
cm = confusion_matrix(y_test, og_y_pred)
print(cm)

[[920  81]
 [214 194]]


>>From the confusion matrix below, we observe that the largest discrepancy occurs in the false negative category, where the model predicted 214 customers would not churn when, in reality, they did churn. This indicates that while the model performs reasonably well overall, its ability to correclty identify actual churners (class 1) needs improvement. Reducing the number of false negatives is critical, as these represent missed opportunities to intervene and retain customers. Addressing this issue could significantly enhance the model's practical utility.

<div style="display: flex; justify-content: space-around;">
    <img src="Graphs/ogdatacm.jpeg" alt="Image 1" width="450">
</div>

__Figure V.__ Confusion matrix of the random forest model without any modifications

>> Before focusing on improving the model's ability to predict churners, I will first run a cross-validation test to ensure that the model generalizes well to unseen data. Cross-validation is done to ensure that the model is not underfitting or overfitting.

>> As can be seen below, the cross-validation accuracy is around 1% higher than the test dataset. This 1% increase can most likely be written off as randomness and accepted as normal variability. Since the test accuracy and the cross-validation accuracy are close, we are able to assume that the model generalizes well to unseen data and is neither underfitting or overfitting.

In [23]:
cv_scores = cross_val_score(og_rf_model, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy:", cv_scores.mean())

Cross-Validation Accuracy: 0.8024496415293279


>> Before analyzing the importance of each feature, I will first attempt to improve the values listed under precision and recall for class 1 in the Classification Report. I do this by ensuring that the ```class_weight``` parameter in the ```RandomForestClassifier()``` is set to ```'balanced'```. This approach assigns greater weight to the minority class, churners, during training. By doing so, the model penalizes misclassifications of churners more heavily than misclassification of non-churners, encouraging the model to correctly predict churners more often. Since churners represent the greatest potential loss of profit, improving the model's ability to idenity them is critical. In theory, this adjustment should improve the recall for class 1 by reducing the number of false negatives, even if it comes at the cost of overall accuracy or precision.

In [24]:
weighted_rf_model = RandomForestClassifier(
    n_estimators= 100,
    max_depth=10,
    min_samples_leaf=4,
    min_samples_split=10,
    random_state=30,
    class_weight='balanced'
)

weighted_rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=4, min_samples_split=10,
                       random_state=30)

>> The updated classification report shows that although the overall accuracy of the model slightly decreased, the metric we care most about -recall for class 1- increased significantly. This is a positive result because it means the model is now better at identifying actual churners, reducing the number of false negatives. As discussed earlier, improving recall for churners aligns with our goal of minimizing porfit loss by identifying customers at risk of leaving. While precision for class 1 did decline, this trade-off is acceptable because the increased recall allows for more actionable retention efforts. ALl this shows that including ```class_weight = 'balanced'``` helped the model become a better predictor of actual churners. Next, I will lower the threshold and look into oversampling techniques.

In [25]:
weighted_y_pred = weighted_rf_model.predict(X_test)
weighted_y_pred_proba = weighted_rf_model.predict_proba(X_test)[:,1]

accuracy = accuracy_score(y_test, weighted_y_pred)
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n {classification_report(y_test, weighted_y_pred)}")
roc_auc = roc_auc_score(y_test, weighted_y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

Accuracy: 0.7693399574166075
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.79      0.83      1001
           1       0.58      0.72      0.64       408

    accuracy                           0.77      1409
   macro avg       0.73      0.75      0.74      1409
weighted avg       0.79      0.77      0.78      1409

ROC-AUC Score: 0.8402834910187852


In [51]:
weighted_cv_scores = cross_val_score(weighted_rf_model, X_train, y_train, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy:", weighted_cv_scores.mean())

Cross-Validation Accuracy: 0.7740498438930751


In [26]:
weighted_cm = confusion_matrix(y_test, weighted_y_pred)
print(weighted_cm)

[[790 211]
 [114 294]]


<div style="display: flex; justify-content: space-around;">
    <img src="Graphs/balancedcm.jpeg" alt="Image 1" width="450">
</div>

__Figure VI.__ Confusion matrix after included ```class_weight='balanced'``` in the Random Forest model

>> Examing the classification report after lowering threshold reveals a couple of things. Precision for non-churners increased, but decreased for churners. Recall decreased for non-churners, but increased for churners significantly. F1-score decreased for non-churners, but incresed for churners. By lowering the threshold, the model has become even better at accurately predicting churners. Even though there was a slight dropoff in some other categories, I believe that the benefits of a 0.09 value increase offsets that, and that lowering the threshold is a good decision to make. It is important to note that we cannot __arbitrarily lower the threshold to an extreme value, such as 0.1__. While this would significantly improve the recall value for churners (class 1), it would drastically reduce precision, resulting in a model that is nearly useless for deriving actionable insights to decrease churn rate. 

__Note.__ Although not explicitly included in this notebook, I created a classification report with the same threshold as shown below, but I removed the ```class_weight='balanced'``` parameter from the Random Forest model. The results showed a __0.20 decrease__ in recall when predicting churners (class 1). This highlights the critical role that  of the ```class_weight``` parameter in addressing class imbalance and improving the model's ability to identify churners. Additionally, lowering the models threshold complements this approach by prioriziting recall, ensuring that more churners are accurately identified. 

>> To figure out exaclty what threshold to use, I set a range starting from 0.1 and increasing incrementally until we reach 0.5. The output shows the different values for recall, precision, and f1-score at different thresholds. Using the F1-score to judge the trade-off between recall and precision, we can see that the optimum value to use as the threshold would be 0.4. This is the value that we will use moving forward when using our model to predict whether future customers are going to churn. 

In [27]:
thresholds = [0.1, 0.2, 0.3, 0.4, 0.5]  # Thresholds to test
for threshold in thresholds:
    y_pred_adjusted = (weighted_y_pred_proba >= threshold).astype(int)
    print(f"Threshold: {threshold}")
    print(f"Recall (Class 1): {recall_score(y_test, y_pred_adjusted):.2f}")
    print(f"Precision (Class 1): {precision_score(y_test, y_pred_adjusted):.2f}")
    print(f"F1-Score: {f1_score(y_test, y_pred_adjusted):.2f}")
    print("-" * 30)

Threshold: 0.1
Recall (Class 1): 0.99
Precision (Class 1): 0.37
F1-Score: 0.54
------------------------------
Threshold: 0.2
Recall (Class 1): 0.93
Precision (Class 1): 0.43
F1-Score: 0.58
------------------------------
Threshold: 0.3
Recall (Class 1): 0.89
Precision (Class 1): 0.48
F1-Score: 0.63
------------------------------
Threshold: 0.4
Recall (Class 1): 0.81
Precision (Class 1): 0.54
F1-Score: 0.65
------------------------------
Threshold: 0.5
Recall (Class 1): 0.72
Precision (Class 1): 0.58
F1-Score: 0.64
------------------------------


In [28]:
threshold = 0.4
weighted_y_pred_adjusted = (weighted_y_pred_proba >= threshold).astype(int)
print(classification_report(y_test, weighted_y_pred_adjusted))

              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1001
           1       0.54      0.81      0.65       408

    accuracy                           0.75      1409
   macro avg       0.72      0.77      0.72      1409
weighted avg       0.80      0.75      0.76      1409



In [29]:
threshold_cm = confusion_matrix(y_test, weighted_y_pred_adjusted)
print(threshold_cm)

[[718 283]
 [ 76 332]]


<div style="display: flex; justify-content: space-around;">
    <img src="Graphs/wltcm.jpeg" alt="Image 1" width="450">
    
</div>

__Figure VII.__ Confusin matrix after including ```class_weight='balanced'``` and lowering the prediction threshold.

>> Another way to potentially improve the recall score for class 1, churners, is by using Synthetic Minority Oversampling Technique (SMOTE). The goal of SMOTE and other resamping techniques is to address the class imbalance in a dataset which would negatively impact a model's ability to correctly predict the minority class. Unlike including the ```class_weight='balanced'``` parameter, which adjusts the model's focus during training by reweighting the classes, SMOTE direclty alters the dataset by generating synthetic samples of the minority class to balance the distribution. As can be seen in the model below, I have removed the ```class_weight``` parameter, as we are assuming that the dataset is not imbalanced anymore. One thing to note here is what is defined as imbalance when it comes to class representation in a dataset. Despite there not being a strict threshold, datasets tend to be considered imbalanced when the minority class __constitues less than 20%__ of the total samples. In the original dataset, the __minority class makes up around 26%__ of the total samples.

In [30]:
smote_rf_model = RandomForestClassifier(
    n_estimators= 100,
    max_depth=10,
    min_samples_leaf=4,
    min_samples_split=10,
    random_state=30,
)


In [31]:
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train, y_train)

smote_rf_model.fit(X_train_resampled, y_train_resampled)

RandomForestClassifier(max_depth=10, min_samples_leaf=4, min_samples_split=10,
                       random_state=30)

In [32]:
smote_y_pred = smote_rf_model.predict(X_test)
smote_y_pred_proba = smote_rf_model.predict_proba(X_test)[:,1]

>> As can be seen from the classification reports below. The __recall value for just resampling is 0.71__ while the recall value for both resampling and then lowering the __decision threshold is 0.79__. Although there definitely is some improvement when incorporating these changes, these recall values are still less than that of combining both ```class_weight='balanced'``` and lowering the threshold value.

In [33]:
print(classification_report(y_test, smote_y_pred))

threshold = 0.4
smote_y_pred_adjusted = (smote_y_pred_proba >= threshold).astype(int)
print(classification_report(y_test, smote_y_pred_adjusted))

              precision    recall  f1-score   support

           0       0.87      0.79      0.83      1001
           1       0.58      0.71      0.64       408

    accuracy                           0.77      1409
   macro avg       0.73      0.75      0.74      1409
weighted avg       0.79      0.77      0.78      1409

              precision    recall  f1-score   support

           0       0.90      0.73      0.81      1001
           1       0.55      0.79      0.65       408

    accuracy                           0.75      1409
   macro avg       0.72      0.76      0.73      1409
weighted avg       0.79      0.75      0.76      1409



In [34]:
smote_cm = confusion_matrix(y_test, smote_y_pred)
print(smote_cm)

smote_threshold_cm = confusion_matrix(y_test, smote_y_pred_adjusted)
print(smote_threshold_cm)

[[795 206]
 [119 289]]
[[735 266]
 [ 86 322]]


<div style="display: flex; justify-content: space-around;">
    <img src="Graphs/SMOTE.jpeg" alt="Image 1" width="300">
    <img src="Graphs/lowerSMOTE.jpeg" alt="Image 2" width="300">
</div>

>> After running going through including ```class_weight='balanced'``` in the Random Forest model, lowering the threshold value, and resampling the data, the combination of techniques that yielded the highest recall value with little sacrifice to the other features was the combination of including ```class_weight``` and lowering the threshold to 0.4. These are the two techniques that I will use moving forward.

>> Before moving onto analyizng what features are important and providng insights using that information as to how to potentially reduce churn, I am first going to hypertune the parameters in the Random Forest model. The purpose of hypertuning the parameters in the Random Forest model is to further optimize the the model's performance. These parameters directly influence how a machine learning algorithm learns patterns and makes predictions. 

>> Using grid search, we can see that the optimum value for ```'n_estimators'```, ```'max_depth'```, ```'min_samples_leaf'```, ```'min_samples_split'``` are 200, 10, 2, and 5, respectively.

In [35]:
param_grid = {
    'n_estimators': [100, 200, 300],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4]
}

rf = RandomForestClassifier()
grid_search = GridSearchCV(rf, param_grid, cv=5, scoring='accuracy', verbose=2, n_jobs=1)
grid_search.fit(X_train, y_train)
best_params = grid_search.best_params_
print(f"Best Parameters: {best_params}")

Fitting 5 folds for each of 108 candidates, totalling 540 fits


[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=100; total time=   0.5s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estimators=200; total time=   0.9s
[CV] END max_depth=None, min_samples_leaf=1, min_samples_split=2, n_estim

>> After hypertuning the parameter and rerunning the weighted random forest model, we see that the parameters are actually not performing as well as we hoped when it comes to the recall value for class 1. This is the aspect of the model that we care most about as it demonstrates the models ability to correctly predict customers that are churners. Even though other metrics may have improve, the values that I used before hypertuning actually showed that the model had a higher recall value. Running the classification report after lowering the threshold also shows that the recall rate is not as high for the hypertuned Random Forest model in comparison to the values that we started out with. Thus, it would make sense to go back to the values that we initially had to maximize the model's ability to correctly predict churners. 

In [36]:
hyper_weighted_rf_model = RandomForestClassifier(
    n_estimators= best_params['n_estimators'],
    max_depth=best_params['max_depth'],
    min_samples_leaf=best_params['min_samples_leaf'],
    min_samples_split=best_params['min_samples_split'],
    random_state=30,
    class_weight='balanced'
)

hyper_weighted_rf_model.fit(X_train, y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_split=10, random_state=30)

In [37]:
hyper_weighted_y_pred = weighted_rf_model.predict(X_test)
hyper_weighted_y_pred_proba = weighted_rf_model.predict_proba(X_test)[:,1]

accuracy = accuracy_score(y_test, hyper_weighted_y_pred)
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n {classification_report(y_test, hyper_weighted_y_pred)}")
roc_auc = roc_auc_score(y_test, hyper_weighted_y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

Accuracy: 0.7693399574166075
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.79      0.83      1001
           1       0.58      0.72      0.64       408

    accuracy                           0.77      1409
   macro avg       0.73      0.75      0.74      1409
weighted avg       0.79      0.77      0.78      1409

ROC-AUC Score: 0.8402834910187852


In [38]:
threshold = 0.4
hyper_weighted_y_pred_adjusted = (weighted_y_pred_proba >= threshold).astype(int)
print(classification_report(y_test, weighted_y_pred_adjusted))

              precision    recall  f1-score   support

           0       0.90      0.72      0.80      1001
           1       0.54      0.81      0.65       408

    accuracy                           0.75      1409
   macro avg       0.72      0.77      0.72      1409
weighted avg       0.80      0.75      0.76      1409



>> Now that we have successfully figured out the best setup for the model, and what threshold has the best recall with minimal trade-off, we can run the feature importance test and see what features to focus on to provide actionable insights. 

>> We use ```feature_importances_``` to calculate the weight that each feature holds during the creation of the random forest, and then rank them by the value of importance. We can see that the top features are 'tenure', and 'TotalCharges'. 

In [39]:
feature_importance = pd.DataFrame(
    {'importance': weighted_rf_model.feature_importances_},
    index = X.columns
)

In [40]:
feature_importance.sort_values(by='importance', ascending=False)

,importance
tenure,0.177070
TotalCharges,0.144340
Contract_Two year,0.095637
MonthlyCharges,0.095514
InternetService_Fiber optic,0.075428
PaymentMethod_Electronic check,0.053285
Contract_One year,0.039816
TechSupport_Yes,0.032807
OnlineSecurity_Yes,0.030276
TotalSubscriptions,0.027809


>> Now that we have a list of features with their corresponding importance values, we can start figuring out which features to keep and which ones we may want to consider getting rid of. One of the main reasons for removing features that we believe to not be contributing to the overall prediction ability of the model, is that it reduces complexity. Reducing complexity is important as it improves interpretability, which allows for easier extrapolation of actionable insights, and reduces training and prediction time, amongst other things. I have decided to remove any features whose value is less than that of the mean value of all feature importance values. This results in removing a good number of the features, as their importance lies below the threshold stated above, and leaves us with 'tenure', 'MonthlyCharges', 'TotalCharges', 'InternetService_Fiber optic', 'StreamingMovies_No internet service', 'StreamingMovies_Yes', 'Contract_One year', 'Contract_Two year', 'PaymentMethod_Electronic check'. The next steps would be to create a dataframe with only these columns, create a new Random Forest model, and test to see how the classification report changes. 

In [54]:
reduced_features = feature_importance[feature_importance['importance'] < feature_importance['importance'].mean()]
important_features = feature_importance[feature_importance['importance'] > feature_importance['importance'].mean()]
print(important_features)

                                importance
tenure                            0.177070
MonthlyCharges                    0.095514
TotalCharges                      0.144340
InternetService_Fiber optic       0.075428
Contract_One year                 0.039816
Contract_Two year                 0.095637
PaymentMethod_Electronic check    0.053285


In [42]:
reduced_X = X.drop(
    ['gender', 'SeniorCitizen', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines',
     'PaperlessBilling', 'TotalSubscriptions', 'InternetService_No', 'OnlineSecurity_No internet service',
     'OnlineSecurity_Yes', 'OnlineBackup_No internet service', 'OnlineBackup_Yes', 'DeviceProtection_No internet service',
     'DeviceProtection_Yes', 'TechSupport_No internet service', 'TechSupport_Yes', 'StreamingTV_No internet service',
     'StreamingTV_Yes', 'PaymentMethod_Credit card (automatic)', 'PaymentMethod_Mailed check'], 
    axis=1 
)

In [43]:
reduced_scaled_X = scaler.fit_transform(reduced_X)

In [44]:
reduced_X_train, reduced_X_test, reduced_y_train, reduced_y_test = train_test_split(reduced_scaled_X, y, test_size=0.2, random_state=30)

In [45]:
reduced_weighted_rf_model = RandomForestClassifier(
    n_estimators= 100,
    max_depth=10,
    min_samples_leaf=4,
    min_samples_split=10,
    random_state=30,
    class_weight='balanced'
)

reduced_weighted_rf_model.fit(reduced_X_train, reduced_y_train)

RandomForestClassifier(class_weight='balanced', max_depth=10,
                       min_samples_leaf=4, min_samples_split=10,
                       random_state=30)

>> From the classification reports below, we can see that removing features whose importance value lay below that of the mean of all importance values, there was a slight 0.01 to 0.02 decrease in values such as recall for class 1 and precision for class 2. On the other hand, there was a similar increase amount for values class 1 precision and class 2 recall, which actually improved from __81% to 83%__. Since class 2 recall is our main target when improving the model, I believe that the trade-offs in this scenario are worth using the reduced Random Forest model moving forward. 

In [46]:
reduced_weighted_y_pred = reduced_weighted_rf_model.predict(reduced_X_test)
reduced_weighted_y_pred_proba = reduced_weighted_rf_model.predict_proba(reduced_X_test)[:,1]

accuracy = accuracy_score(reduced_y_test, reduced_weighted_y_pred)
print(f"Accuracy: {accuracy}")
print(f"Classification Report:\n {classification_report(reduced_y_test, reduced_weighted_y_pred)}")
roc_auc = roc_auc_score(reduced_y_test, reduced_weighted_y_pred_proba)
print(f"ROC-AUC Score: {roc_auc}")

Accuracy: 0.7672107877927609
Classification Report:
               precision    recall  f1-score   support

           0       0.87      0.79      0.83      1001
           1       0.58      0.72      0.64       408

    accuracy                           0.77      1409
   macro avg       0.73      0.75      0.73      1409
weighted avg       0.79      0.77      0.77      1409

ROC-AUC Score: 0.8343482497894263


In [47]:
threshold = 0.4
reduced_weighted_y_pred_adjusted = (reduced_weighted_y_pred_proba >= threshold).astype(int)
print(classification_report(reduced_y_test, reduced_weighted_y_pred_adjusted))

              precision    recall  f1-score   support

           0       0.91      0.70      0.79      1001
           1       0.53      0.83      0.65       408

    accuracy                           0.74      1409
   macro avg       0.72      0.77      0.72      1409
weighted avg       0.80      0.74      0.75      1409



In [50]:
reduced_cv_scores = cross_val_score(reduced_weighted_rf_model, X_train, reduced_y_train, cv=5, scoring='accuracy')
print("Cross-Validation Accuracy:", reduced_cv_scores.mean())

Cross-Validation Accuracy: 0.7740498438930751


>## Analysis

| **Aspect**                     | **Details**                                                                                  |
|--------------------------------|----------------------------------------------------------------------------------------------|
| **Best Model Configuration**   | `reduced_weight_rf_model`                                                                    |
| **Key Features Identified**    | `important_features`                                                                         |
| **Most Important Feature**     | `tenure`                                                                                     |
| **Insight from Tenure**        | Long-time subscribers are significantly less likely to churn.                                |
| **Second Most Important Feature** | `TotalCharges`                                                                             |
| **Third Most Important Feature** | `Contract_Two year`                                                                         |
| **Recommended Action 1**       | Reduce subscription prices for new customers to encourage long-term retention.               |
| **Recommended Action 2**       | Implement a rewards program for loyal customers to offset feelings of unfairness.            |
| **Justification**              | Rewards programs, like those in casino hotels, incentivize both new and loyal customers.     |
| **Considerations**             | Long-time customers may feel betrayed if new subscribers pay less. Rewards programs can mitigate this. |
| **Figure Reference**           | __Figure II__: Shows the drastic reduction in churn likelihood with increased tenure.        |

>> After all of our tests, figuring out which Random Forest model configuration is best, and what features to remove, we have come to the conclusion that ```reduced_weight_rf_model``` is the Random Forest setup that has the highest accuracy at predicting recall (churners) with the smallest trade-off of other prediction values. We have also deduced that the training data does not need to include all of the features from the original data set, and only needs to include the features listed in ```important_features```. 

>> From ```important_features``` we can see that ```tenure``` is the number one most important feature used when determining whether someone is going to churn. As stated at the beginning of this project, there is no direct way to influence the amount of time someone has spent subscribed with the company. However, there is still something to be taken from this. I believe that the most important interpretation of this feature is that it tells us that once someone has stayed subscribed with the company for a period of time, the chances that they wil suddenly churn drops drastically. This is something also shown in __Figure II__. Since ```TotalCharges``` and ```Contract_Two year``` are the second and third most important features, the first actoinable insight I would reccomend would be to figure out some way to decrease the price of subscribers who are relatively new to the company, and then adjusting the price back to normal after their tenure with the company has fallen into a range that shows they are unlikely to churn. This problem with this is that long time customers may feel betrayed about the fact that new subscribers pay less then they do, despite them having remained loyal. To offset some of these feelings of betrayal, we may need to generate some kind of rewards program -similar to that which casino hotels give out- to loyal customres who have stayed with the company for certain period of time. This not only incentives new customers to stay with the program longer so that they could exprience the reward system, but also appeases customers who have been subscribed for a long time. One important thing to note is that the subscription cost for new customers will eventually rise to that which long tenured customers are paying. 